# Predicting Flight Delays

# 0. SETTINGS

## 0-1. Import packages and modules

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import math
%matplotlib inline

#font configuration
import matplotlib.font_manager as fm
font_location = "/Library/Fonts/AppleGothic.ttf"
font_name = fm.FontProperties(fname=font_location).get_name()
mpl.rc('font', family=font_name)


## 0-2. Import CSV files

In [2]:
data_flights = pd.read_csv('./flights.csv')
data_airplanes = pd.read_csv('./airlines.csv')
data_airports = pd.read_csv('./airports.csv')

# 1. Preparation of Data

## 1-0. Data Inspection

## 1-1. TimeSetting
1. data_flights data의 날짜 Year, Month, Day를 한 feature로 만듭니다.
    - 연/월/일 형태
2. data_flights data의 시각정보인 `SCHEDULED_DEPARTURE`, `DEPARTURE_TIME`을 시각 표시인 **00:00** 로 변경합니다.

1. data_flights의 `YEAR_MONTH_DAY` column 생성

In [ ]:
data_flights['YEAR_MONTH_DAY'] = pd.to_datetime(data_flights[['YEAR', 'MONTH', 'DAY']], format='%y/%m/%d')

2. `SCHEDULED_DEPARTURE` 과 `DEPARTURE_TIME` 을 **00:00** 시각표시로 변경
    - 시각 data의 format 을 변경해주는 함수 `converting_timeformat`

In [ ]:
def converting_timeformat(series):
    """
    data 상에 시각 표현을 00:00:00 형태로 변경하는 함수입니다.
    parameter: type: series. format 을 변경하고 싶은 dataframe의 series를 넣어줍니다.
    return : type: pandas.core.series.Series
    """
    converted_timelist = []
    for i in range(series.shape[0]):
        if math.isnan(series[i]):
            converted_timelist.append(math.nan)
        else:
            padding = "{:04d}".format(int(series[i]))
            #24:00 이면, 00:00 로 바꿔줍니다. 날짜 처리 얘기해봐야됩니다.
            if padding == "2400":
                padding = "0000"
            new_time_format = datetime.time(int(padding[0:2]), int(padding[2:4]))
            converted_timelist.append(new_time_format)
    
    return pd.Series(converted_timelist)
    

In [ ]:
data_flights['formatted_SCHEDULED_DEPARTURE'] = converting_timeformat(data_flights['SCHEDULED_DEPARTURE'])

In [ ]:
data_flights['formatted_DEPARTURE_TIME'] = converting_timeformat(data_flights['DEPARTURE_TIME'])

## 1-2 누락값 처리
- data_flights의 `DEPARTURE_DELAY`의 평균으로 NaN 값 매꾸기
- 이것에 대한 정당성은 이야기해봐야 될 것 같습니다.
- 1. 없애거나 2. 다른 통계량으로 채워넣거나, 3. 다른 방법론으로 추정하거나

In [7]:
mean_of_departure_delay = data_flights['DEPARTURE_DELAY'].mean()
#mean_of_departure_delay = 9.370158275198389
#'DEPARTURE_DELAY'의 NaN 값을 mean_of_departure_delay 로 변경
data_flights['DEPARTURE_DELAY'].fillna(mean_of_departure_delay)

0          -11.0
1           -8.0
2           -2.0
3           -5.0
4           -1.0
5           -5.0
6           -6.0
7           14.0
8          -11.0
9            3.0
10          -6.0
11          -8.0
12           0.0
13          -6.0
14          -1.0
15          -4.0
16         -14.0
17          -6.0
18          -4.0
19          -5.0
20          25.0
21          -8.0
22          -1.0
23          -2.0
24          -3.0
25          -3.0
26          -8.0
27          12.0
28         -10.0
29          21.0
           ...  
5819049      2.0
5819050      2.0
5819051     16.0
5819052    -15.0
5819053      0.0
5819054     -5.0
5819055     -2.0
5819056     -3.0
5819057     13.0
5819058     -7.0
5819059     -8.0
5819060     -5.0
5819061     -3.0
5819062      6.0
5819063      2.0
5819064     -3.0
5819065     -7.0
5819066     -2.0
5819067     -8.0
5819068     -1.0
5819069    159.0
5819070      0.0
5819071     16.0
5819072      7.0
5819073     16.0
5819074     -4.0
5819075     -4.0
5819076     -9

In [8]:
#DEPARTURE_DELAY 를 통해 DEPARTURE_TIME 의 NaN 값 처리
data_flights['DEPARTURE_TIME'] = data_flights['SCHEDULED_TIME'] + data_flights['DEPARTURE_DELAY']